In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import cv2
import numpy as np

c:\Users\prabh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\prabh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\prabh\AppData\Local\Programs\Python\Python39\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [38]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

dataset = datasets.ImageFolder('D:/amogh/python_projects/ML/Mask_detection/dataset', transform=transform)
                                

In [39]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [40]:
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True,num_workers=2,pin_memory=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=2,pin_memory=True)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [42]:
next(iter(train_data))[0].shape

torch.Size([32, 3, 224, 224])

In [43]:
model=nn.Sequential(nn.Conv2d(3,16,3),nn.ReLU(),nn.MaxPool2d(2,2),nn.Conv2d(16,32,3),nn.ReLU(),nn.MaxPool2d(2,2),nn.Conv2d(32,16,3),nn.ReLU(),nn.MaxPool2d(2,2),nn.Flatten(),nn.Linear(16*26*26,256),nn.ReLU(),nn.BatchNorm1d(256),nn.Dropout(p=0.5),nn.Linear(256,2)).to(device)
# model=nn.Sequential(nn.Conv2d(3,16,5),nn.ReLU(),nn.MaxPool2d(4,4),nn.Conv2d(16,8,5),nn.ReLU(),nn.MaxPool2d(4,4),nn.Flatten(),nn.Linear(8*12*12,256),nn.ReLU(),nn.BatchNorm1d(256),nn.Dropout(p=0.5),nn.Linear(256,2))
criterion = nn.CrossEntropyLoss()
# optimiser = torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-2)

In [44]:
from fastai.optimizer import OptimWrapper
from torch import optim

In [45]:
# def opt_func(params, **kwargs): return OptimWrapper(torch.optim.Adam(params,lr=0.001,weight_decay=1e-2))
from functools import partial
opt_func = partial(OptimWrapper, opt=torch.optim.Adam)

from fastai.data.core import DataLoaders
dls = DataLoaders(train_data, test_data)

from fastai.learner import Learner
from fastai.callback.progress import ProgressCallback

# from fastai.callback.data import CudaCallback

learn = Learner(dls, model, loss_func=criterion, opt_func=opt_func)

In [46]:
learn.fit(20)

epoch,train_loss,valid_loss,time
0,0.298506,0.236451,00:30
1,0.230851,0.201727,00:30
2,0.224184,0.223974,00:29
3,0.227519,0.403179,00:29
4,0.194858,0.202201,00:29
5,0.193322,0.191457,00:29
6,0.173350,0.180735,00:29
7,0.177377,0.174224,00:29
8,0.152518,0.200904,00:29
9,0.149348,0.232862,00:29


In [47]:
from torchsummary import summary
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 222, 222]             448
              ReLU-2         [-1, 16, 222, 222]               0
         MaxPool2d-3         [-1, 16, 111, 111]               0
            Conv2d-4         [-1, 32, 109, 109]           4,640
              ReLU-5         [-1, 32, 109, 109]               0
         MaxPool2d-6           [-1, 32, 54, 54]               0
            Conv2d-7           [-1, 16, 52, 52]           4,624
              ReLU-8           [-1, 16, 52, 52]               0
         MaxPool2d-9           [-1, 16, 26, 26]               0
          Flatten-10                [-1, 10816]               0
           Linear-11                  [-1, 256]       2,769,152
             ReLU-12                  [-1, 256]               0
      BatchNorm1d-13                  [-1, 256]             512
          Dropout-14                  [

In [48]:
# epochs=2
# for epoch in range(epochs):
#     cor=0
#     for (imgs,labels) in train_data:
#         imgs = imgs.to(device)
#         labels=labels.to(device)
#         output=model(imgs)
#         loss=criterion(output,labels)
#         for param in model.parameters():
#             param.grad=None
#         loss.backward()
#         optimiser.step()

        
#         _,predicted = torch.max(output.data,1)
#         cor+=(predicted == labels).sum().item()
#         # acc=sum(output.round()==y_train)/len(y_train)
#     print(epoch,cor)

In [59]:
FILE='model_pth'
torch.save(model.state_dict(),FILE)

In [50]:

import matplotlib.pyplot as plt
torch.cuda.memory_allocated(0)

49672192

In [51]:
model.eval()
with torch.no_grad():
    num_img=0
    cor=0
    for (imgs,labels) in test_data:
        imgs = imgs.to(device)
        labels=labels.to(device)
        output=model(imgs)
        _,predicted = torch.max(output.data,1)
        cor+=(predicted == labels).sum().item()
        num_img+=predicted.shape[0]
        
        # image=imgs[0]
        # plt.imshow(transforms.ToPILImage()(image))
        # plt.show()
        # print(predicted[0].item(),labels[0].item())
        # cv2.imshow("",np.array(image)*255)
        # cv2.waitKey(0)

    print(cor/num_img*100)

94.74969474969474


In [52]:
import cv2

In [53]:
#defining prototext and caffemodel paths
caffeModel = "D:/amogh/python_projects/ML/Face_Detection/Face-detection-with-OpenCV-and-deep-learning-master/models/res10_300x300_ssd_iter_140000.caffemodel"
prototextPath = "D:/amogh/python_projects/ML/Face_Detection/Face-detection-with-OpenCV-and-deep-learning-master/models/deploy.prototxt.txt"

#Load Model
print("Loading model...................")
net = cv2.dnn.readNetFromCaffe(prototextPath,caffeModel)

Loading model...................


In [57]:
dic={0:"MASK",1:"NO MASK"}
colour_dic={0:(0,255,0),1:(0,0,255)}

In [58]:
# initialize the video stream to get the video frames
print("[INFO] starting video stream...")
camera=cv2.VideoCapture(0)

while True :
    
    _,image=camera.read()
    
    (h, w) = image.shape[:2]
    try:
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

        net.setInput(blob)
        detections = net.forward()

        # Identify each face
        for i in range(0, detections.shape[2]):
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            confidence = detections[0, 0, i, 2]

            # If confidence > 0.5, save it as a separate file
            if (confidence > 0.5):
                frame = image[startY-50:endY+50, startX-50:endX+50]
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # show the output frame
                test_img = np.moveaxis(frame,2,0)
            
                transform = transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            ])
                test_img=transform(torch.FloatTensor(test_img))/255
                test_imgs = torch.utils.data.DataLoader([test_img], batch_size=1, shuffle=True,num_workers=0)
                
               
                with torch.no_grad():
                    img=next(iter(test_imgs))
                    # for imgs in test_imgs:
                        # pass
                        # imgs = imgs.to(device)
                    output=model(img.to(device))
                    _,predicted = torch.max(output.data,1)
                    # image=imgs[0]
                    # plt.imshow(transforms.ToPILImage()(image))
                    # plt.show()
                    # print(predicted)
                
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                
                cv2.rectangle(image, (startX, startY), (endX, endY),
                            colour_dic[predicted.item()], 2)
                cv2.putText(image, dic[predicted.item()]+'    '+str(text), (startX, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.45, colour_dic[predicted.item()], 2)
    except:pass
    cv2.imshow("Frame", image)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
camera.release()
cv2.destroyAllWindows()


[INFO] starting video stream...
